##팔당댐(ARIMA로..?)
######현재수위 : 보간, 시간차 적용 해야함
######공용량   : 보간, 크롤링해온 후 시간차 적용 해야함
######방류량   : 보간, 시간차 적용
######/content/drive/MyDrive/신입기수 프로젝트/Code/유찬/팔당댐_보간_코드.ipynb
######/content/drive/MyDrive/신입기수 프로젝트/Code/세정/시간차 적용_ 팔당댐.ipynb
######유입량   : 버림
######저수량   : 버림


---


##강화대교 조위
######ARIMA로 결측치 처리
######/content/drive/MyDrive/신입기수 프로젝트/Code/예림/강화대교 조위 전처리_2022.08.16.ipynb
######시간차 적용
######/content/drive/MyDrive/신입기수 프로젝트/Code/세정/시간차 적용 2일치.ipynb


---


##유량
######청담대교 : train data는 곡선 fit, test data는 일단 보간(arima로 바꾸자, 다 arima로 채운거랑 비교해보자)
######잠수교   : 버림
######한강대교 : 보간(arima로 바꾸자)
######행주대교 : 보간(arima로 바꾸자)
######/content/drive/MyDrive/신입기수 프로젝트/Code/다영/청담대교_유량_curve_fit.ipynb


---


##수위(ARIMA로..?)
######청담대교 : 보간
######잠수교   : 보간
######한강대교 : 보간
######행주대교 : 보간




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize']=(10,10)

import warnings
warnings.filterwarnings(action='ignore')
import time

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten 
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D 
from keras.callbacks import ModelCheckpoint, EarlyStopping

#GPU 자원이 부족
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
## 1. 데이터 불러오기, 공통 전처리
raw_data = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')

###유량 처리
유량 = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/잠수교_유량_수위_보간/청담대교_cuve_fit.csv', encoding='cp949')
유량 = 유량.iloc[:,[2,4,6,8]]
유량 = 유량.interpolate()
_raw_data = pd.concat((raw_data.drop(유량.columns,axis=1), 유량),axis=1)
raw_data = _raw_data

###제거
raw_data_ = raw_data.drop(raw_data[['유입량','저수량','대곡교_강수량','진관교_강수량','송정동_강수량','잠수교_유량']], axis = 1)

###강화대교 조위 시간차
강화대교_조위 = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/시간차/강화대교_조위 for 전체.csv', encoding='cp949')
raw_data__ = pd.concat((raw_data_.drop(강화대교_조위.columns,axis=1),강화대교_조위),axis=1)

###방류량 조위 시간차
방류량 = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/시간차/방류량 for 전체.csv', encoding='cp949')
__raw_data = pd.concat((raw_data__.drop(방류량.columns,axis=1),방류량),axis=1)
raw_data___ = __raw_data.set_index('날짜',drop=True)

###보간 처리
보간처리 = raw_data___[['현재수위','공용량','방류량','강화대교_조위', '청담대교_유량', '한강대교_유량', '행주대교_유량', '청담대교_수위', '잠수교_수위', '한강대교_수위', '행주대교_수위']].interpolate(method = 'linear' , limit_direction = 'forward')
보간처리df = pd.concat([raw_data___.drop(보간처리.columns, axis = 1), 보간처리], axis=1)

#train / test data 생성
idx = []
for i in range(12):
  idx.append(i*26496)

In [4]:
보간처리df

,현재수위,공용량,방류량,강화대교_조위,청담대교_유량,한강대교_유량,행주대교_유량,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_수위
날짜,,,,,,,,,,,
2012-05-01 00:00,24.800,24.93,575.70,193.0,469.05,729.80,540.18,310.7,300.2,290.0,275.3
2012-05-01 00:10,24.794,25.15,575.30,199.0,498.00,731.48,540.18,314.7,300.2,290.0,275.3
2012-05-01 00:20,24.789,25.31,555.00,207.0,490.68,726.42,540.18,313.7,301.2,290.0,275.3
2012-05-01 00:30,24.789,25.31,575.70,215.0,476.21,726.42,552.17,311.7,301.2,290.0,276.3
2012-05-01 00:40,24.789,25.31,575.40,226.0,476.21,707.17,564.29,311.7,301.2,291.0,277.3
...,...,...,...,...,...,...,...,...,...,...,...
2022-07-18 23:10,25.040,31.14,258.67,407.0,319.84,-456.41,974.40,0.0,0.0,0.0,0.0
2022-07-18 23:20,25.040,31.14,258.58,431.0,314.01,-717.30,1006.88,0.0,0.0,0.0,0.0
2022-07-18 23:30,25.040,31.14,258.56,453.0,387.55,-843.37,1039.90,0.0,0.0,0.0,0.0


In [ ]:
start = time.time()

y_predict = []

ytest_list = sorted(glob("/content/drive/MyDrive/신입기수 프로젝트/Dataset/submission 임시/*.csv"))  #정답 데이터

ytest = []

for i in range(len(ytest_list)):
  tmp = pd.read_csv(ytest_list[i], encoding = 'cp949')
  ytest.append(np.array(tmp.iloc[:,1])*100)


xdata = []
ydata = []
xdata_idx = []
ydata_idx = [-4,-3,-2,-1]
for i in range(len(보간처리df.columns)-4):
    xdata_idx.append(i)

w =  432 #window size

for i in tqdm(range(11)):
    if i != 10:
        tmp = 보간처리df[idx[i]:idx[i+1]]
    else:
        tmp = 보간처리df[idx[i]:]

    for j in range(len(tmp)-w+1):
        xdata.append(np.array(tmp.iloc[j:j + w, xdata_idx]).astype(float))

    for j in range(w-1,len(tmp)):
        ydata.append(np.array(tmp.iloc[j, ydata_idx]).astype(float))

xdata_ = np.array(xdata)
ydata_ = np.array(ydata)

xtrain = xdata_[:-6912]
ytrain = ydata_[:-6912]
xtest = xdata_[-6912:]
#ytest = ydata_[-6912:]
    
# 모델링 및 모델 학습

# !pip install keras 

""" KERAS IMPLEMENTATION """

# Research on parameter : Dilation_Rate of the convolution 
if 'model' in locals(): 
    del model 
if 'history' in locals():
    del history

model = Sequential()
model.add(Conv1D(filters = 32,
                input_shape = (w, len(보간처리df.columns)-4),
                data_format = 'channels_last',
                kernel_size = 2,
                activation = 'relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters = 12,
                data_format = 'channels_last',
                kernel_size = 2,
                activation = 'relu'))

model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(45, activation = 'relu'))
model.add(Dense(4))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=10),
                keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                                monitor='val_loss',
                                                save_best_only=True)]

history = model.fit(xtrain, ytrain, epochs=300, verbose=1, validation_split=0.2, callbacks=callbacks) #기본은 500
plt.plot(history.history['loss'])
plt.show()

keras_model_best = keras.models.load_model('best_model.h5')
keras_model_best.evaluate(xtest,ytest)
y_predict.append(keras_model_best.predict(xtest))

print("time :", time.time() - start, "초")

100%|██████████| 11/11 [02:24<00:00, 13.16s/it]


In [ ]:
tl_predict = np.array(y_predict).reshape(6912,4).T

In [ ]:
f, axes = plt.subplots(4, 1)
f.set_size_inches((20, 35))
plt.subplots_adjust(wspace = 0.3, hspace = 0.3)

for i in range(4):
    axes[i].plot(ytest[i], marker = ",", label = "actual")
    axes[i].plot(tl_predict[i], marker = ",", label="predict")
    axes[i].legend(prop={'size':10})  

plt.show()

In [ ]:
#제출 파일 생성

sample_submission = pd.read_csv("/content/drive/MyDrive/신입기수 프로젝트/DACON/competition_data/sample_submission.csv")

sample_submission["wl_1018662"] = tl_predict[0]
sample_submission["wl_1018680"] = tl_predict[1]
sample_submission["wl_1018683"] = tl_predict[2]
sample_submission["wl_1019630"] = tl_predict[3]

sample_submission.to_csv("conv1d 전처리.csv", index = False)